# Análisis de datos y estadísticas

#### Top 10 de palabras que se encuentran primero de todas las partidas

#### Gráfico que muestre el porcentaje de partidas por estado.

In [ ]:
# uso de matplotlib e informacion_partidas.csv//columna[2] "Estado"

#### Gráfico que muestre el porcentaje de partidas finalizadas según género

In [ ]:
# uso de matplotlib e informacion_partidas.csv//columna[8] "Genero"

#### Gráfico que muestre el procentaje de partidas finalizadas según nivel

In [ ]:
# uso de matplotlib e informacion_partidas.csv//columna[7] "Nivel"
#mismo procesamiento que gráfico anterior, cambia columna

#### Duración total de la partida

In [ ]:
# no estaría entendiendo si acá deberíamos implementar un gráfico, una suma total de todas las duraciones o cómo hacer este procesamiento.

#### Tiempo de la respuesta exitosa más rápida

In [ ]:
# debería el informar máximo de (timestamp de estado "OK" - timestamp de registro anterior) 

#### Promedio de tiempo de respuesta para respuestas exitosas

In [ ]:
# sum(columna estado "OK") / len (columna estado)

#### Promedio de tiempo de partida general

In [ ]:
# variable que suma (timestamp(estado inicio_partida) - timestamp(estado fin)) y divide por cant de partidas [sobre esto último deberíamos consultar si se considerará también las canceladas]

#### Porcentaje de aciertos por usuarie

#### Cantidad de tarjetas para las que el usuarie no dio respuesta

#### Cantidad de tarjetas en las que el usuarie dio una respuesta errónea

## Opcional

#### Gráfico que muestre el porcentaje de partidas que se juegan para cada dia de la semana

#### Promedio de tiempo de partidas finalizadas por nivel

#### Porcentaje de palabras encontradas en las partidas "timeout"